## we will be using a dataset containing bone marrow transplantation characteristics for pediatric patients from UCI’s Machine Learning Repository.
### Steps for creating the Machine Learning Model includes
    1. importing the necessary libraries
    2. Loading and explore the dataset
    3. Model building 
    4. Communicating results
    

In [1]:
# importing the necessary libraries
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA


from sklearn.metrics import confusion_matrix

from scipy.io import arff

### 2. Explore

In [5]:
data = arff.loadarff('bone-marrow.arff')
df = pd.DataFrame(data[0])

In [6]:
df.head()

,Recipientgender,Stemcellsource,Donorage,Donorage35,IIIV,Gendermatch,DonorABO,RecipientABO,RecipientRh,ABOmatch,...,extcGvHD,CD34kgx10d6,CD3dCD34,CD3dkgx10d8,Rbodymass,ANCrecovery,PLTrecovery,time_to_aGvHD_III_IV,survival_time,survival_status
0,b'1',b'1',22.830137,b'0',b'1',b'0',b'1',b'1',b'1',b'0',...,b'1',7.20,1.338760,5.38,35.0,19.0,51.0,32.0,999.0,0.0
1,b'1',b'0',23.342466,b'0',b'1',b'0',b'-1',b'-1',b'1',b'0',...,b'1',4.50,11.078295,0.41,20.6,16.0,37.0,1000000.0,163.0,1.0
2,b'1',b'0',26.394521,b'0',b'1',b'0',b'-1',b'-1',b'1',b'0',...,b'1',7.94,19.013230,0.42,23.4,23.0,20.0,1000000.0,435.0,1.0
3,b'0',b'0',39.684932,b'1',b'1',b'0',b'1',b'2',b'1',b'1',...,b'?',4.25,29.481647,0.14,50.0,23.0,29.0,19.0,53.0,1.0
4,b'0',b'1',33.358904,b'0',b'0',b'0',b'1',b'2',b'0',b'1',...,b'1',51.85,3.972255,13.05,9.0,14.0,14.0,1000000.0,2043.0,0.0


In [7]:
df.shape

(187, 37)

In [ ]:
data = arff.loadarff('bone-marrow.arff')
df = pd.DataFrame(data[0])
df.drop(columns=['Disease'], inplace=True)


#Convert all columns to numeric, coerce errors to null values
for c in df.columns:
    df[c] = pd.to_numeric(df[c], errors='coerce')
    
#Make sure binary columns are encoded as 0 and 1
for c in df.columns[df.nunique()==2]:
    df[c] = (df[c]==1)*1.0

# 1. Calculate the number of unique values for each column
print(f'Count of unique values in each column:{ df.columns}')

# 2. Set target, survival_status,as y; features (dropping survival status and time) as X


# 3. Define lists of numeric and categorical columns based on number of unique values


# 4. Print columns with missing values


# 5. Split data into train/test split


# 6. Create categorical preprocessing pipeline
# Using mode to fill in missing values and OHE


# 7. Create numerical preprocessing pipeline
# Using mean to fill in missing values and standard scaling of features


# 8. Create column transformer that will preprocess the numerical and categorical features separately

# 9. Create a pipeline with preprocess, PCA, and a logistic regresssion model


# 10. Fit the pipeline on the training data

#Predict the pipeline on the test data


# 11. Define search space of hyperparameters

#12. Search over hyperparameters abolve to optimize pipeline and fit


# 13. Save the best estimator from the gridsearch and print attributes and final accuracy on test set


# 14. Print attributes of best_model


# 15. Print final accuracy score 
